In [5]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id="797b22135893422ab981597948963843",
                                               client_secret="80f412c2e1fa4a6999d52767d3d5fc7b",
                                               redirect_uri="http://127.0.0.1:7146/callback",
                                               scope="user-library-read"))



In [6]:


taylor_uri = 'spotify:artist:06HL4z0CvFAxyc27GXpf02'

results = sp.artist_albums(taylor_uri, album_type='album')

albums = results['items']
while results['next']:
    results = sp.next(results)
    albums.extend(results['items'])

for album in albums:
    print(album['name'])

THE TORTURED POETS DEPARTMENT: THE ANTHOLOGY
THE TORTURED POETS DEPARTMENT
1989 (Taylor's Version) [Deluxe]
1989 (Taylor's Version)
Speak Now (Taylor's Version)
Midnights (The Til Dawn Edition)
Midnights (3am Edition)
Midnights
Red (Taylor's Version)
Fearless (Taylor's Version)
evermore (deluxe version)
evermore
folklore: the long pond studio sessions (from the Disney+ special) [deluxe edition]
folklore (deluxe version)
folklore
Lover
Taylor Swift Karaoke: reputation
reputation
reputation Stadium Tour Surprise Song Playlist
1989 (Deluxe Edition)
1989
Red (Deluxe Edition)
Red
Speak Now World Tour Live
Speak Now (Deluxe Edition)
Speak Now
Fearless Platinum Edition
Fearless
Live From Clear Channel Stripped 2008
Taylor Swift


In [13]:
taylor_uri = 'spotify:artist:06HL4z0CvFAxyc27GXpf02'
taylor_ID = '06HL4z0CvFAxyc27GXpf02'

print(sp.artist(taylor_uri))
print(sp.artist_related_artists(taylor_ID))


HTTP Error for GET to https://api.spotify.com/v1/artists/06HL4z0CvFAxyc27GXpf02/related-artists with Params: {} returned 404 due to Not Found


{'external_urls': {'spotify': 'https://open.spotify.com/artist/06HL4z0CvFAxyc27GXpf02'}, 'followers': {'href': None, 'total': 135932753}, 'genres': [], 'href': 'https://api.spotify.com/v1/artists/06HL4z0CvFAxyc27GXpf02', 'id': '06HL4z0CvFAxyc27GXpf02', 'images': [{'url': 'https://i.scdn.co/image/ab6761610000e5ebe672b5f553298dcdccb0e676', 'height': 640, 'width': 640}, {'url': 'https://i.scdn.co/image/ab67616100005174e672b5f553298dcdccb0e676', 'height': 320, 'width': 320}, {'url': 'https://i.scdn.co/image/ab6761610000f178e672b5f553298dcdccb0e676', 'height': 160, 'width': 160}], 'name': 'Taylor Swift', 'popularity': 98, 'type': 'artist', 'uri': 'spotify:artist:06HL4z0CvFAxyc27GXpf02'}


SpotifyException: http status: 404, code: -1 - https://api.spotify.com/v1/artists/06HL4z0CvFAxyc27GXpf02/related-artists:
 Not Found, reason: None